In [ ]:
# Import libraries
import requests
import pandas as pd
import pandas_gbq as bq
import pydata_google_auth



In [ ]:
import json
import pandas as pd
import requests

url = "https://api.hubapi.com/marketing-emails/v1/emails/"
all_objects = []
offset = 0
limit = 10  # This seems to be the default limit from the API

headers = {
    'accept': "application/json",
    'authorization': f'Bearer {key}'
}

try:
    # Loop until we get all records
    while True:
        # Add offset to querystring
        querystring = {
            "properties": ["campaign"],
            "offset": offset
        }
        
        response = requests.request("GET", url, headers=headers, params=querystring)
        response.raise_for_status()
        data = response.json()
        
        objects = data['objects']
        all_objects.extend(objects)
        
        print(f"Fetched {len(objects)} objects. Total so far: {len(all_objects)}")
        
        # Check if we've got all records
        if len(all_objects) >= data['total']:
            break
            
        # Update offset for next batch
        offset += limit

    print(f"\nTotal records fetched: {len(all_objects)}")

    # Create list of dictionaries for DataFrame
    campaign_pairs = []
    for obj in all_objects:
        campaign = obj.get('campaign', '')
        campaign_ids = obj.get('allEmailCampaignIds', [])
        
        if campaign_ids:
            for email_id in campaign_ids:
                campaign_pairs.append({
                    'campaign_id': campaign,
                    'email_campaign_id': email_id,
                    'campaign_name': obj.get('campaignName', '')
                })

    # Convert to DataFrame and save
    df = pd.DataFrame(campaign_pairs)
    print(f"Created DataFrame with {len(df)} rows")
    
    df.to_csv("emailid_w_campaignid.csv", index=False)
    print("Data successfully exported to emailid_w_campaignid.csv")

except requests.exceptions.RequestException as e:
    print(f"Error making API request: {e}")
except json.JSONDecodeError as e:
    print(f"Error parsing JSON response: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
df = pd.read_csv('emailid_w_campaignid.csv')

In [ ]:
url = "https://api.hubapi.com/marketing/v3/campaigns/"
querystring = {
    "properties": ["hs_campaign_status", "hs_end_date", "hs_notes", "hs_name", "hs_start_date"]
}

# Set headers with the authorization token
headers = {
    'accept': "application/json",
    'authorization': f'Bearer {key}'  # Replace {key} with your actual token
}

# Make the GET request
response = requests.request("GET", url, headers=headers, params=querystring)

# Check if the response is successful
if response.status_code == 200:
    data = response.json()  # Parse the JSON response
    campaigns = data.get("results", [])  # Get the campaigns list if available

    # Create a list to store campaign data
    campaign_data = []

    for campaign in campaigns:
        campaign_data.append({
            # campaignid
            "GuiID": campaign.get("id", "No ID"),
            "Campaign Name": campaign.get("properties", {}).get("hs_name", "Unnamed Campaign"),
            "Status": campaign.get("properties", {}).get("hs_campaign_status", "Unknown Status"),
            "Start Date": campaign.get("properties", {}).get("hs_start_date", "No Start Date"),
            "End Date": campaign.get("properties", {}).get("hs_end_date", "No End Date"),
            "Notes": campaign.get("properties", {}).get("hs_notes", "No Notes"),
        })

    # Create a DataFrame from the campaign data
    df = pd.DataFrame(campaign_data)

    # Print the DataFrame
    print(df)

    # Optionally save the DataFrame to a CSV file
    df.to_csv("campaign_data.csv", index=False)
else:
    print(f"Error: {response.status_code} - {response.text}")

In [ ]:
df = pd.read_csv('campaign_data.csv')
# project_id = 'x-marketing'
# bq.to_gbq(df, 'data_catalog.sample_records', project_id, if_exists='replace', credentials=credentials)
df


Connect to bigquery

In [ ]:
SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform',
    'https://www.googleapis.com/auth/drive',
]

credentials = pydata_google_auth.get_user_credentials(
    SCOPES,
    # Note, this doesn't work if you're running from a notebook on a
    # remote sever, such as over SSH or with Google Colab. In those cases,
    # install the gcloud command line interface and authenticate with the
    # `gcloud auth application-default login` command and the `--no-browser`
    # option.
    auth_local_webserver=True,
)

In [ ]:
project_id = 'x-marketing'
bq.to_gbq(df, 'sdi.email_campaign', project_id, if_exists='replace', credentials=credentials)

In [ ]:
project_id = 'x-marketing'
bq.to_gbq(df, 'sdi.campaign', project_id, if_exists='replace', credentials=credentials)